# 自定义层

深度学习成功背后的一个因素是神经网络的灵活性：
我们可以用创造性的方式组合不同的层，从而设计出适用于各种任务的架构。
例如，研究人员发明了专门用于处理图像、文本、序列数据和执行动态规划的层。
未来，你会遇到或要自己发明一个现在在深度学习框架中还不存在的层。
在这些情况下，你必须构建自定义层。在本节中，我们将向你展示如何构建。

## 不带参数的层

首先，我们(**构造一个没有任何参数的自定义层**)。
如果你还记得我们在 :numref:`sec_model_construction`对块的介绍，
这应该看起来很眼熟。
下面的`CenteredLayer`类要从其输入中减去均值。
要构建它，我们只需继承基础层类并实现前向传播功能。


In [21]:
import torch
import torch.nn.functional as F
from torch import nn

# 自定义一个层和自定义一个网络是没有任何区别的
# 它们都是 nn.Module 的子类
# 这是一个不带参数的层
class CenteredLayer(nn.Module):
    def __init__(self):
        # 如果不使用 super().__init__()
        # 那么 Pyhton 也会默认加上它的调用
        super().__init__()

    def forward(self, X):
        # 把均值变为 0
        return X - X.mean()

让我们向该层提供一些数据，验证它是否能按预期工作。


In [22]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

现在，我们可以[**将层作为组件合并到构建更复杂的模型中**]。


In [23]:
# 把自己创建的层作为 "组件" 合并到复杂的网络模型中
# 这和 PyTorch 中定义的那些层的类，用起来没有本质的区别
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

作为额外的健全性检查，我们可以在向该网络发送随机数据后，检查均值是否为0。
由于我们处理的是浮点数，因为存储精度的原因，我们仍然可能会看到一个非常小的非零数。


In [24]:
Y = net(torch.rand(4, 8))
Y.mean() # 3.7253e-09 是一个非常小的数值，浮点数计算有误差，视为 0

tensor(-9.3132e-10, grad_fn=<MeanBackward0>)

## [**带参数的层**]

以上我们知道了如何定义简单的层，下面我们继续定义具有参数的层，
这些参数可以通过训练进行调整。
我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。
比如管理访问、初始化、共享、保存和加载模型参数。
这样做的好处之一是：我们不需要为每个自定义层编写自定义序列化程序。

现在，让我们实现自定义版本的全连接层。
回想一下，该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。
在此实现中，我们使用修正线性单元作为激活函数。
该层需要输入参数：`in_units`和`units`，分别表示输入和输出的数量。


In [25]:
# 定义自己的线性层
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        # 初始化参数 weight And Bias
        # 不同的是，为了能够让自定义的参数传给 优化函数 进行优化
        # 所以要使用 Parameter 来定义自己的参数
        # 在访问 Parameter 定义的参数的时候要使用 .data 的方式来访问数据
        
        # 猜测：==> 如何通过 PyTorch 访问OR注册一层的参数
        # 因为 Python 是知道每个 self 中的变量的类型的
        # 所以 PyTorch 框架可以通过 Python 的这个特性
        # 来确定哪些 self 的成员变量是参数，是需要更新的
        # 也就是知道了哪些成员变量是需要传给 优化函数的
        
        # 通过 nn.Parameter 来对 tensor 进行一个包装就可以了
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
        
    def forward(self, X):
        # 使用 .data 来访问具体的参数的数据
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

接下来，我们实例化`MyLinear`类并访问其模型参数。


In [26]:
linear = MyLinear(5, 3)
# 访问 weight 参数
linear.weight

Parameter containing:
tensor([[-1.7935,  0.3346, -0.9824],
        [ 1.7462, -0.5608, -1.2286],
        [-0.6448,  2.1250,  0.9079],
        [-0.7753, -1.0932,  0.4266],
        [ 0.9041,  0.3312,  0.0273]], requires_grad=True)

我们可以[**使用自定义层直接执行前向传播计算**]。


In [27]:
linear(torch.rand(2, 5)) # 前向传播

tensor([[0.1749, 0.5659, 1.2345],
        [0.1125, 0.5539, 0.4208]])

我们还可以(**使用自定义层构建模型**)，就像使用内置的全连接层一样使用自定义层。


In [28]:
# 将自定义的带参数的 "层" 作为组件放进网络中进行 "合并"
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

## 小结

* 我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与深度学习框架中的任何现有层不同。
* 在自定义层定义完成后，我们就可以在任意环境和网络架构中调用该自定义层。
* 层可以有局部参数，这些参数可以通过内置函数创建。

## 练习

1. 设计一个接受输入并计算张量降维的层，它返回$y_k = \sum_{i, j} W_{ijk} x_i x_j$。
1. 设计一个返回输入数据的傅立叶系数前半部分的层。


[Discussions](https://discuss.d2l.ai/t/1835)
